## Part 1

In [ ]:
from astroquery.gaia import Gaia
import astropy.units as u

# Define the position of Ruprecht 147, search radius, and Gaia G magnitude threshold
ra_r147 = 289.074 * u.deg
dec_r147 = -16.323 * u.deg
r_search = 1 * u.deg
G_thresh = 14

# ADQL query for Gaia DR3 sources within 1° of Ruprecht 147 (G < 14), crossmatched with 2MASS
query = f"""
SELECT *
FROM gaiadr3.gaia_source AS g
JOIN gaiadr3.tmass_psc_xsc_best_neighbour AS xmatch USING (source_id)
JOIN gaiadr3.tmass_psc_xsc_join AS xjoin USING (clean_tmass_psc_xsc_oid)
JOIN gaiadr1.tmass_original_valid AS tmass
    ON xjoin.original_psc_source_id = tmass.designation
WHERE
    CONTAINS(
        POINT('ICRS', g.ra, g.dec),
        CIRCLE('ICRS', 289.074, -16.323, 1)
    ) = 1
    AND g.phot_g_mean_mag <= 14
"""

# Launch the query asynchronously
job = Gaia.launch_job_async(query=query)
gaiadr3_match = job.get_results()

# Preview the first few rows
print(gaiadr3_match[:5])


INFO: Query finished. [astroquery.utils.tap.core]
    solution_id             DESIGNATION          ... ext_key    j_date   
                                                 ...              d      
------------------- ---------------------------- ... ------- ------------
1636148068921376768 Gaia DR3 4088892600474423808 ...      --  2450952.755
1636148068921376768 Gaia DR3 4088893154546972672 ...      --  2450952.755
1636148068921376768 Gaia DR3 4088893188906716032 ...      --  2450952.755
1636148068921376768 Gaia DR3 4087815667506086656 ...      -- 2450952.7656
1636148068921376768 Gaia DR3 4087815697559746304 ...      -- 2450952.7656


## Part 2

In [14]:
# Determine how many stars were returned from the query
n_stars = len(gaiadr3_match)
print(f"Number of stars within 1° of Ruprecht 147 with G ≤ {G_thresh}: {n_stars}")

Number of stars within 1° of Ruprecht 147 with G ≤ 14: 3637


## Part 3